In [22]:
import gzip
import os
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

In [21]:
sample_data = pd.read_csv('../data/other/OSD-524-samples.csv') # Change this sample file name to get the corrsponding sample csv
sample_name_list = np.unique(sample_data['Sample Name'])
sample_data.shape

(41, 21)

In [23]:
sample_data.columns

Index(['Source Name', 'Sample Name', 'Characteristics: Organism',
       'Characteristics: Strain', 'Characteristics: Genotype',
       'Characteristics: Material Type', 'Factor Value: Ionizing Radiation',
       'Factor Value: Generation', 'Protocol REF', 'Protocol REF.1',
       'Parameter Value: ionizing radiation energy',
       'Parameter Value: exposure duration',
       'Parameter Value: absorbed radiation dose',
       'Parameter Value: absorbed radiation dose rate',
       'Parameter Value: ionizing radiation categorized by source',
       'Protocol REF.2', 'Parameter Value: Sample Preservation Method',
       'Parameter Value: Sample Storage Temperature',
       'Parameter Value: Age at time of sample collection',
       'Comment: Animal Source', 'Comment: Parental Treatment'],
      dtype='object')

In [24]:
directory = '../data/raw/gz'
notation_list = []
dna_list = []
mass_list = []
filename_list = []
label_list = []
LIMIT = 3000
label_name = 'Factor Value: Generation'

if LIMIT:
    LIMIT = LIMIT - LIMIT % 4 - 1

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.gz'):
        for name in sample_name_list:
            if name in filename:
                label = sample_data[sample_data['Sample Name'] == name][label_name].values[0]
                break
        with gzip.open(os.path.join(directory, filename), 'rb') as file:
            count = 0
            c = 0
            for line in file:
                line = line.decode('utf-8')
                line = line.replace('\n', '')
                if count % 4 == 0:
                    notation_list.append(line[1:])
                if count % 4 == 1:
                    dna_list.append(line)
                if count % 4 == 3:
                    mass_list.append(line[1:])
                    filename_list.append(filename)
                    label_list.append(label)
                    c += 1
                count += 1
                if LIMIT and c > LIMIT:
                    break

print(len(notation_list), len(dna_list), len(mass_list), len(filename_list))

100%|██████████| 2/2 [00:00<00:00, 57.13it/s]

6000 6000 6000 6000


In [28]:
data = pd.DataFrame({
    'notation': notation_list,
    'DNA': dna_list,
    'mass': mass_list,
    'filename': filename_list,
    'label': label_list
})
print(data.shape)

temp = input("Enter a value for your data processing, make sure the value is consistent during the whole process: ")
temp.replace(" ", "")


data.to_csv(f'../data/intermediate/{temp}_dna_data.csv', index=False)

(6000, 5)


In [26]:
data.head(5)

,notation,DNA,mass,filename,label
0,SRR5756263.1 1 length=150,NTTATATAAGATAAAGGTTTTAATTGTATTAAGTGATAGTTAAGTC...,"<AAAKKKKAAKKKAFKAFFAKKKKAKKKF,FF,AKFF,,FFKKKKF...",GLDS-524_wgbs_GSM2684058_R1_raw.fastq.gz,F1
1,SRR5756263.2 2 length=150,NTTTTAAATGTGAATATATTTGATTTTTTTTGGTTAATTTGATTTG...,"AAFFFKKKKFKKKKKKKKKKKKKFFKKKK7AFAKKKK,FKKKKKKK...",GLDS-524_wgbs_GSM2684058_R1_raw.fastq.gz,F1
2,SRR5756263.3 3 length=150,NAATTAAAATTATAAATATTGGAATTAAGAATATTTAAGTGTTCGG...,"AAFFKKKKKKKKKKAKAKKKFFAKFKKKKKKKAKKKFK7KKKKK,,...",GLDS-524_wgbs_GSM2684058_R1_raw.fastq.gz,F1
3,SRR5756263.4 4 length=150,NATTATAATTTATTGTAGTATTTTGGAGAAATGTGTTATTATTGAA...,AAF<FKKKKKKKKKKKKKAKFKKKAFKKKKKA77FKKKKKKKKKKK...,GLDS-524_wgbs_GSM2684058_R1_raw.fastq.gz,F1
4,SRR5756263.5 5 length=150,NTAGGAGTGTTGGTAGATGTGATATAAATTAAAGGCATTATTATTA...,"AAFFFK,F<FFFFKKKKKAFKKKKKKKKKFKKKK,AAFKFFKKFKA...",GLDS-524_wgbs_GSM2684058_R1_raw.fastq.gz,F1


In [27]:
# Label proportion
from collections import Counter
my_list = list(data['label'])
element_count = Counter(my_list)
total_elements = len(my_list)
element_percentages = {key: (count / total_elements) * 100 for key, count in element_count.items()}
for key, percentage in element_percentages.items():
    print(f"{key}: {percentage:.2f}%")

F1: 100.00%


In [ ]:
# # Check
# count = 5
# with gzip.open('../data/GLDS-524_wgbs_GSM2684058_R1_raw.fastq.gz', 'rb') as file:
#     for line in file:
#         line = line.decode('utf-8')
#         print(line)
#         count -= 1
#         if count < 1:
#             break